In [1]:
import os
import sys
import time
import galsim
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib import patches
from PIL import Image 

from skimage import io, img_as_uint

In [2]:
# Useful when viewing fullsize images through the VNC
save_dir = "/g/g15/ahlquist/Documents/workspace/output_dir"

In [3]:
# Directory where data is found
data_dir = "/usr/workspace/DSSI_ch/data_and_information"

In [4]:
# Shared folder
team_6_dir = "/usr/workspace/DSSI_ch/team6"
npz_index = 0 

## load data

In [5]:
npz_data = np.load(f"{data_dir}/asteroid_equalized_images_{npz_index}.npz")
starAsteroid = npz_data['data']

In [6]:
y = np.load(f"{data_dir}/asteroid_injected_information_{npz_index}.npz", allow_pickle=True)
X = starAsteroid

In [ ]:
#Information of the first sample image from the Zwicky Transient Facility
y['data'][0]

In [ ]:
#Information of the magnitude, length, angle and box of the injected asteroid streaks
print(y['data'])
y['data'][1]

In [ ]:
box_values_ast_0 = y['data'][1]['box'][1]
plt.imshow(X[0][box_values_ast_0.ymin:box_values_ast_0.ymax,box_values_ast_0.xmin:box_values_ast_0.xmax])

In [ ]:
def displayAsteroid(asteroid, image):
    box_values = asteroid
    plt.imshow(image[box_values.ymin+40:box_values.ymax-40,box_values.xmin+40:box_values.xmax-40])
    
displayAsteroid(y['data'][5]['box'][1], X[2])

## Generate images from npz file

In [ ]:
""" Generate .jpg files from npz
(Don't run, unless you want to overwrite files)
"""

for i in tqdm(range(len(X))):
    io.imsave(f"{team_6_dir}/data/astroids_{npz_index}_{i}.jpg", img_as_uint(X[i]))

In [7]:
""" Generate .csv files from npz
(Don't run, unless you want to overwrite files)
"""

df = pd.DataFrame(columns=["xmin", "xmax", "ymin", "ymax"])

for i in tqdm(range(1, len(y['data']), 2)):
    
    # Get wanted filename index
    index = int(i/2)
    
    # Add all boxes to dataframe
    for _,row in y['data'][i]['box'].iteritems():
        df = df.append({"xmin": row.xmin, "xmax": row.xmax, 
                   "ymin": row.ymin, "ymax": row.ymax}, ignore_index=True)
        
    # Save dataframe to file
    df.to_csv(f"{team_6_dir}/data/astroids_{npz_index}_{index}.csv", index=False)
    
    # Clear dataframe
    df = df.iloc[0:0]

100%|██████████| 100/100 [00:09<00:00, 10.21it/s]


In [8]:
""" For each image make a set of box csv where astroids do NOT exist
"""
df = pd.DataFrame(columns=["xmin", "xmax", "ymin", "ymax"])
for i in tqdm(range(1, len(y['data']), 2)):
    
    # Get wanted filename index
    index = int(i/2)
    
    ast_df = pd.read_csv(f"{team_6_dir}/data/astroids_{npz_index}_{index}.csv")

    image = X[index]
    height = len(image)
    width = len(image[0])

    rng_x = np.random.default_rng()
    rng_y = np.random.default_rng()

    num_points = 64
    
    # Generate random center points:
    ixs = rng_x.integers(51, width-51, dtype=np.int32, size=num_points + 20, endpoint=False)
    iys = rng_y.integers(51, height-51, dtype=np.int32, size=num_points + 20, endpoint=False)
    
    ixs_min = ixs - 51
    ixs_max = ixs + 51
    iys_min = iys - 51
    iys_max = iys + 51

    # duplicate rows to make a 2D array of size ix, 
    # such that operations between the two can be done.
    
    ast_xmin = np.tile(ast_df["xmin"], (len(ixs), 1)).T
    ast_xmax = np.tile(ast_df["xmax"], (len(ixs), 1)).T
    ast_ymin = np.tile(ast_df["ymin"], (len(iys), 1)).T
    ast_ymax = np.tile(ast_df["ymax"], (len(iys), 1)).T

    mask = (ast_xmin <= ixs_max) & (ast_xmax >= ixs_min) \
         & (ast_ymin <= iys_max) & (ast_ymax >= iys_min)

    # Get all indexs where any overlap exists  
    conflict_mask = mask.any(axis=0)
    valid_mask = ~conflict_mask 

    df["xmin"] = ixs_min[valid_mask]
    df["xmax"] = ixs_max[valid_mask]
    df["ymin"] = iys_min[valid_mask]
    df["ymax"] = iys_max[valid_mask]
    
    df = df.head(num_points)

    # Save dataframe to file
    df.to_csv(f"{team_6_dir}/data/not_astroids_{npz_index}_{index}.csv", index=False)

    # Clear dataframe
    df = df.iloc[0:0]
    

100%|██████████| 100/100 [00:00<00:00, 139.48it/s]


In [9]:
""" Check asteroid boxes to see if they are in the black spaces (remove if they are)
"""

def isValidBoundingBox(b):

    cutoff = 130
    
    sliced = image[b["ymin"]:b["ymax"], b["xmin"]:b["xmax"]]

    mean = np.mean(sliced)
    sliced = Image.fromarray(sliced, 'L')
    sliced = sliced.resize((400, 400))

    #print(mean)
    #if(mean < cutoff):
        #sliced = sliced.save(f"{save_dir}/geeks.jpg")
        #print(f"ymin: {b['ymin']}, ymax: {b['ymax']}, xmin: {b['xmin']}, xmax:{b['xmax']}")
        #display(sliced)
        #if(b['ymin'] == 1841 and b['ymax'] == 1849 and b['xmin'] == 2023 and b['xmax'] == 2031):
        #    sliced = sliced.save(f"{save_dir}/geeks.png")

    return mean > cutoff

In [10]:
df = pd.DataFrame()

for i in tqdm(range(1, len(y['data']), 2)):
    
    # Get wanted filename index
    index = int(i/2)
    
    boxes = pd.read_csv(f"{team_6_dir}/data/astroids_{npz_index}_{index}.csv")
    image = Image.open(f"{team_6_dir}/data/astroids_{npz_index}_{index}.jpg").convert("L")

    image = np.array(image)
    
    offset = 47
    
    df["xmin"] = boxes["xmin"] + offset
    df["ymin"] = boxes["ymin"] + offset
    df["xmax"] = boxes["xmax"] - offset
    df["ymax"] = boxes["ymax"] - offset
    
    mask = df.apply(lambda b : isValidBoundingBox(b), axis=1)
    df = boxes.loc[mask]
    
    # Save dataframe to file
    # Overwrite:
    df.to_csv(f"{team_6_dir}/data/astroids_{npz_index}_{index}.csv", index=False)
    # NewFile:
    #df.to_csv(f"{team_6_dir}/data/astroids_cleaned_{npz_index}_{index}.csv", index=False)
    
    # Clear dataframe
    df = df.iloc[0:0]

  2%|▏         | 2/100 [00:00<00:14,  6.95it/s]

ymin: 2962, ymax: 2971, xmin: 1186, xmax:1195
ymin: 2950, ymax: 2959, xmin: 2664, xmax:2673


  9%|▉         | 9/100 [00:01<00:11,  7.95it/s]

ymin: 1394, ymax: 1402, xmin: 2256, xmax:2264
ymin: 1935, ymax: 1943, xmin: 1525, xmax:1533


 11%|█         | 11/100 [00:01<00:10,  8.74it/s]

...

 95%|█████████▌| 95/100 [00:11<00:00,  8.40it/s]

ymin: 2008, ymax: 2017, xmin: 687, xmax:696
ymin: 1216, ymax: 1223, xmin: 1548, xmax:1555


 98%|█████████▊| 98/100 [00:12<00:00,  7.37it/s]

ymin: 518, ymax: 527, xmin: 2954, xmax:2963
ymin: 2358, ymax: 2366, xmin: 598, xmax:606
ymin: 2279, ymax: 2287, xmin: 829, xmax:837


100%|██████████| 100/100 [00:12<00:00,  8.09it/s]


In [ ]:
""" Generate tensorflow.csv containing all bounding boxes
filename,width,height,class,xmin,ymin,xmax,ymax
"""
# Generate tensorflow.csv containing all bounding boxes
#filename,width,height,class,xmin,ymin,xmax,ymax

df = pd.DataFrame(columns=["filename", "width", "height", "class", "xmin", "xmax", "ymin", "ymax"])

# Possibly add for-loop of other npz files
for i in range(1, len(y['data']), 2):
    
    # Get wanted filename index
    index = int((i+1)/2)
    
    img_file = f"{team_6_dir}/data/astroids_{npz_index}_{index}.jpg"
    
    if not os.path.exists(img_file):
        continue
    
    width, height = Image.open(img_file).size
    
    # Add all boxes in image to dataframe
    for _,row in y['data'][i]['box'].iteritems():
        df = df.append({
                "filename": img_file,
                "width": width, "height": height, 
                "class": "AST",
                "xmin": row.xmin, "xmax": row.xmax, 
                "ymin": row.ymin, "ymax": row.ymax}, ignore_index=True)
        
# Save dataframe to file
df.to_csv(f"{team_6_dir}/data/tensorflow.csv", index=False)

In [ ]:
""" View some files with Ground Truth boxes labeled
"""
index = 0
i = index * 2 + 1
#for i in tqdm(range(1, len(y['data']), 2)):
    
# Get wanted filename index
index = int(i/2)
image = X[index]

fig = plt.figure(figsize=(40, 40))

#add axes to the image
ax = fig.add_axes([0,0,1,1])
plt.imshow(image, cmap="Greys_r")

ground_truth = y['data'][i]['box']
for box in ground_truth:  
    xmin = box.xmin
    ymin = box.ymin
    xmax = box.xmax
    ymax = box.ymax

    width = xmax - xmin
    height = ymax - ymin

    # assign different color to different classes of objects  
    edgecolor = 'lime'
    ax.annotate('Ast', xy=(xmax-40,ymin+20))

    # add bounding boxes to the image
    rect = patches.Rectangle((xmin,ymin), width, height, edgecolor = edgecolor, facecolor = 'none', lw=4)
    ax.add_patch(rect)


plt.savefig(f"{save_dir}/asteroid_ground_truths_{index}", format='png')